In [1]:
import pandas as pd
import pyspark.sql.functions as F

<hr />

## srag 2019

<hr />

In [4]:
srag_2019 = spark.read.csv('gs://ai-covid19-datalake/raw/srag/2019/srag_2019.csv', inferSchema=True, header=True, sep=';')

In [10]:
# columns types according to srag dictionary
# date_type: 
date_type_columns = ['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_NASC', 'DT_VGM', 'DT_RT_VGM', 'DT_UT_DOSE', 'DT_VAC_MAE', 'DT_DOSEUNI', 
                     'DT_1_DOSE', 'DT_2_DOSE', 'DT_ANTIVIR', 'DT_ENTUTI', 'DT_SAIDUTI', 'DT_RAIOX', 'DT_COLETA', 'DT_EVOLUCA', 'DT_DIGITA']

# categorical_type:
categorical_type_columns = ['SG_UF_NOT', 'ID_MUNICIP', 'CO_MUN_NOT', 'ID_REGIONA', 'CO_REGIONA', 'CS_SEXO', '']



srag_2019.printSchema()

root
 |-- DT_NOTIFIC: string (nullable = true)
 |-- SEM_NOT: string (nullable = true)
 |-- DT_SIN_PRI: string (nullable = true)
 |-- SEM_PRI: string (nullable = true)
 |-- SG_UF_NOT: string (nullable = true)
 |-- ID_REGIONA: string (nullable = true)
 |-- CO_REGIONA: string (nullable = true)
 |-- ID_MUNICIP: string (nullable = true)
 |-- CO_MUN_NOT: string (nullable = true)
 |-- ID_UNIDADE: string (nullable = true)
 |-- CO_UNI_NOT: string (nullable = true)
 |-- CS_SEXO: string (nullable = true)
 |-- DT_NASC: string (nullable = true)
 |-- NU_IDADE_N: integer (nullable = true)
 |-- TP_IDADE: integer (nullable = true)
 |-- COD_IDADE: integer (nullable = true)
 |-- CS_GESTANT: integer (nullable = true)
 |-- CS_RACA: integer (nullable = true)
 |-- CS_ETINIA: string (nullable = true)
 |-- CS_ESCOL_N: integer (nullable = true)
 |-- ID_PAIS: string (nullable = true)
 |-- CO_PAIS: integer (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- CO_RG_RESI: integer (nullable = true)
 |-- ID_MN

In [9]:
srag_2019.limit(5).toPandas()

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,DT_DIGITA,HISTO_VGM,PAIS_VGM,CO_PS_VGM,LO_PS_VGM,DT_VGM,DT_RT_VGM,PCR_SARS2,PAC_COCBO,PAC_DSCBO
0,10/01/2019,2,06/01/2019,2,MG,BELO HORIZONTE,1449,BELO HORIZONTE,310620,UNIDADE DE PRONTO ATENDIMENTO LESTE,...,14/03/2019,0,None,None,None,None,None,None,None,None
1,03/01/2019,1,01/01/2019,1,SP,GVE I CAPITAL,1331,SAO PAULO,355030,HOSP MUN M BOI MIRIM,...,None,0,None,None,None,None,None,None,None,None
2,02/01/2019,1,31/12/2018,1,PE,1,1497,RECIFE,261160,IMIP,...,None,0,None,None,None,None,None,None,None,None
3,10/01/2019,2,07/01/2019,2,SP,GVE XVII CAMPINAS,1342,CAMPINAS,350950,HOSPITAL MUNICIPAL DR MARIO GATTI CAMPINAS,...,None,0,None,None,None,None,None,None,None,None
4,11/01/2019,2,06/01/2019,2,PE,4,1499,CARUARU,260410,SECRETARIA MUNICIPAL DE SAUDE DE CARUARU,...,None,0,None,None,None,None,None,None,None,None
